<a href="https://colab.research.google.com/github/pA1nD/course-deep-learning/blob/master/L4_Recurrent_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networks (RNN) with Keras

Recurrent neural networks (RNN) are a class of neural networks that is powerful for modeling sequence data such as time series or natural language.

Schematically, a RNN layer uses a `for` loop to iterate over the timesteps of a sequence, while maintaining an internal state that encodes information about the timesteps it has seen so far.

The Keras RNN API is designed with a focus on:

- **Ease of use**: the built-in `tf.keras.layers.RNN`, `tf.keras.layers.LSTM`, `tf.keras.layers.GRU` layers enable you to quickly build recurrent models without having to make difficult configuration choices.
  
- **Ease of customization**: You can also define your own RNN cell layer (the inner part of the `for` loop) with custom behavior, and use it with the generic `tf.keras.layers.RNN` layer (the `for` loop itself). This allows you to quickly prototype different research ideas in a flexible way with minimal code.
  

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

import numpy as np
from sklearn.model_selection import train_test_split

## Predict the next number: [1,2,3,4,5] -> [6]

In [0]:
def create_sequence(n):
  
  data = [[[(i+j)] for i in range(5)] for j in range(n)]
  target = [(i+5) for i in range(n)]

  data = np.array(data, dtype=int)
  target = np.array(target, dtype=int)
  return train_test_split(data, target, test_size=0.2, random_state = 4)

amount = 50000
x_train, x_test, y_train, y_test = create_sequence(amount)

x_train, x_test = x_train / amount, x_test / amount
y_train, y_test = y_train / amount, y_test / amount

# Layers
inputs = tf.keras.Input(shape=(5,1))
rnn = tf.keras.layers.SimpleRNN(2)
outputs = rnn(inputs)

# SimpleRNN(2) would create two outputs. Here we add an additional layer to 
# inputs = tf.keras.Input(shape=(5,1))
# rnn = tf.keras.layers.SimpleRNN(2)
# outputs = rnn(inputs)
# outputs = tf.keras.layers.Dense(1, activation="sigmoid")(outputs)


model = tf.keras.Model(inputs, outputs, name='SimpleRnnModel2')

# magic. print the parameters of the rnn layer.
#weights = rnn.variables
#plt.imshow(weights[0].numpy())


model.compile(loss="mae", optimizer='adam', metrics=['accuracy'])
model.summary()


In [0]:
  history = model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test))

In [0]:
x_test1 = x_test[0:20]
pred = model.predict(x_test1)
print(pred)

In [0]:
import matplotlib.pyplot as plt
plt.scatter(range(20), pred*amount, c='r')
plt.scatter(range(20), y_test[0:20]*amount, c='g')

In [0]:
print(model.metrics_names)

loss, accuracy = model.evaluate(x_test,y_test)

print("loss:", loss, "| accuracy:",accuracy)

## Build a simple model


There are three built-in RNN layers in Keras:

1. `tf.keras.layers.SimpleRNN`, a fully-connected RNN where the output from previous timestep is to be fed to next timestep.

2. `tf.keras.layers.GRU`, first proposed in [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078).

3. `tf.keras.layers.LSTM`, first proposed in [Long Short-Term Memory](https://www.bioinf.jku.at/publications/older/2604.pdf).

In early 2015, Keras had the first reusable open-source Python implementations of LSTM and GRU.

### Load MNIST dataset

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
sample, sample_label = x_train[0], y_train[0]

### Build Model

We'll use as input sequences the sequence of rows of MNIST digits (treating each row of pixels as a timestep), and we'll predict the digit's label.

In [0]:
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(None, 28)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')]
)

model.summary()

### Create a model instance and compile it
We choose `sparse_categorical_crossentropy` as the loss function for the model. The output of the model has shape of `[batch_size, 10]`. The target for the model is a integer vector, each of the integer is in the range of 0 to 9.

In [0]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='sgd',
              metrics=['accuracy'])


In [0]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=100,
          epochs=5)

## IMDB Sentiment

In [0]:
max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

model = tf.keras.models.Sequential([
    Embedding(max_features, 128, input_length=maxlen),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')   
])

# try using different optimizers and different optimizer configs
model.compile('adam', 'mse', metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[x_test, y_test])

## License

Copyright 2019 The TensorFlow Authors and 2020 Björn Schmidtke for GSERM.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.